In [1]:
import cv2
import numpy as np

In [2]:
PATH2DATA = 'C:/Users/abram/Documents/books/Kopytov/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation'

Load image

In [3]:
def get_img():
    img = cv2.imread(PATH2DATA + '/train/Original/2_A.png')
    return cv2.resize(img, (512, 512))

def get_anio():
    img = cv2.imread("C://Users/abram/Downloads/angio-6/slice.png",cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(512,512))
    cv2.imshow('orig',img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(15,15))
    img = clahe.apply(img)
    img = cv2.GaussianBlur(img,(7,7),1.4)
    edges = cv2.Canny(img,30,105)
    cv2.imshow('anio',edges)
    cv2.waitKey(0)

def get_green():
    img = get_img()
    green = img.copy()
    green[:,:,0] = 0
    green[:,:,2] = 0
    return green

#get_anio()

Green channel extraction

In [64]:
img2draw = get_img()
green = img2draw.copy()
green[:,:,0] = 0
green[:,:,2] = 0
cv2.imshow('green', green)
cv2.waitKey()

-1

Porposed method

In [4]:
green = get_green()
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(15,15))
lab = cv2.cvtColor(green,cv2.COLOR_BGR2Lab)
lab_planes = list(cv2.split(lab))
lab_planes[0] = clahe.apply(lab_planes[0])
lab = cv2.merge(lab_planes)
green_clahe = cv2.cvtColor(lab,cv2.COLOR_Lab2BGR)
# cv2.imshow('green_clahe',green_clahe)
# cv2.imshow('green',green)
# cv2.waitKey(0)

In [20]:
gray_green = cv2.cvtColor(green_clahe,cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray_green, (7, 7), 1.4, 1.4)
cv2.imshow('gg',blurred)
cv2.waitKey(0)

-1

In [ ]:
blurred = cv2.GaussianBlur(green_clahe, (5, 5), 1.4, 1.4)
gx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)  # Градиент по оси X
gy = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)  # Градиент по оси Y
M = cv2.magnitude(gx, gy)
angle = np.atan(gy/gx)


In [11]:
def non_maximum_suppression(gradient_magnitude, gradient_direction):
    # Получаем размеры изображения
    height, width = gradient_magnitude.shape
    # Создаем пустой массив для хранения подавленных значений
    suppressed = np.zeros((height, width), dtype=np.float32)

    # Преобразуем углы в градусы
    angle = gradient_direction * 180.0 / np.pi
    angle[angle < 0] += 180

    for y in range(1, height - 1):
        for x in range(1, width - 1):
            # Определяем направление градиента
            q = 255
            r = 255

            # Угол 0 градусов
            if (0 <= angle[y, x] < 22.5) or (157.5 <= angle[y, x] <= 180):
                q = gradient_magnitude[y, x + 1]
                r = gradient_magnitude[y, x - 1]
            # Угол 45 градусов
            elif (22.5 <= angle[y, x] < 67.5):
                q = gradient_magnitude[y + 1, x - 1]
                r = gradient_magnitude[y - 1, x + 1]
            # Угол 90 градусов
            elif (67.5 <= angle[y, x] < 112.5):
                q = gradient_magnitude[y + 1, x]
                r = gradient_magnitude[y - 1, x]
            # Угол 135 градусов
            elif (112.5 <= angle[y, x] < 157.5):
                q = gradient_magnitude[y - 1, x - 1]
                r = gradient_magnitude[y + 1, x + 1]

            # Непосредственное подавление
            if gradient_magnitude[y, x] >= q and gradient_magnitude[y, x] >= r:
                suppressed[y, x] = gradient_magnitude[y, x]

    return suppressed


In [12]:
def my_canny(img,lower,upper):
    blurred = cv2.GaussianBlur(img, (7, 7), 1.4, 1.4)
    green = blurred[:,:,1]
    gx = cv2.Sobel(green, cv2.CV_64F, 1, 0, ksize=3)  # Градиент по оси X
    gy = cv2.Sobel(green, cv2.CV_64F, 0, 1, ksize=3)  # Градиент по оси Y

    magnitude = cv2.magnitude(gx, gy)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    cv2.imshow('Sobel Magnitude', magnitude)

    M, angle = cv2.cartToPolar(gx, gy, angleInDegrees = True)
    height, width, _ = img.shape
    supressed = non_maximum_suppression(M,angle)
    cv2.imshow('supressed', supressed)
    cv2.waitKey(0)
    
    
    
my_canny(green_clahe,0,0)

In [36]:
import numpy as np
import os
import cv2


# defining the canny detector function 

# here weak_th and strong_th are thresholds for 
# double thresholding step 
def Canny_detector(img, weak_th = None, strong_th = None):

    # conversion of image to grayscale 
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Noise reduction step 
    img = cv2.GaussianBlur(img, (9, 9), 1.4)

    # Calculating the gradients 
    gx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0, 3)
    gy = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1, 3)

    # Conversion of Cartesian coordinates to polar 
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees = True)

    # setting the minimum and maximum thresholds 
    # for double thresholding 
    mag_max = np.max(mag)
    if not weak_th:weak_th = mag_max * 0.1
    if not strong_th:strong_th = mag_max * 0.5

    # getting the dimensions of the input image 
    height, width = img.shape

    # Looping through every pixel of the grayscale 
    # image 
    for i_x in range(width):
        for i_y in range(height):

            grad_ang = ang[i_y, i_x]
            grad_ang = abs(grad_ang-180) if abs(grad_ang)>180 else abs(grad_ang)

            # selecting the neighbours of the target pixel 
            # according to the gradient direction 
            # In the x axis direction 
            if grad_ang<= 22.5:
                neighb_1_x, neighb_1_y = i_x-1, i_y
                neighb_2_x, neighb_2_y = i_x + 1, i_y

            # top right (diagonal-1) direction 
            elif grad_ang>22.5 and grad_ang<=(22.5 + 45):
                neighb_1_x, neighb_1_y = i_x-1, i_y-1
                neighb_2_x, neighb_2_y = i_x + 1, i_y + 1

            # In y-axis direction 
            elif grad_ang>(22.5 + 45) and grad_ang<=(22.5 + 90):
                neighb_1_x, neighb_1_y = i_x, i_y-1
                neighb_2_x, neighb_2_y = i_x, i_y + 1

            # top left (diagonal-2) direction 
            elif grad_ang>(22.5 + 90) and grad_ang<=(22.5 + 135):
                neighb_1_x, neighb_1_y = i_x-1, i_y + 1
                neighb_2_x, neighb_2_y = i_x + 1, i_y-1

            # Now it restarts the cycle 
            elif grad_ang>(22.5 + 135) and grad_ang<=(22.5 + 180):
                neighb_1_x, neighb_1_y = i_x-1, i_y
                neighb_2_x, neighb_2_y = i_x + 1, i_y

            # Non-maximum suppression step 
            if width>neighb_1_x>= 0 and height>neighb_1_y>= 0:
                if mag[i_y, i_x]<mag[neighb_1_y, neighb_1_x]:
                    mag[i_y, i_x]= 0
                    continue

            if width>neighb_2_x>= 0 and height>neighb_2_y>= 0:
                if mag[i_y, i_x]<mag[neighb_2_y, neighb_2_x]:
                    mag[i_y, i_x]= 0

    weak_ids = np.zeros_like(img)
    strong_ids = np.zeros_like(img)
    ids = np.zeros_like(img)
    
   # cv2.imshow('non max',mag)
    
    # double thresholding step 
    for i_x in range(width):
        for i_y in range(height):

            grad_mag = mag[i_y, i_x]
            
            if grad_mag<weak_th:
                mag[i_y, i_x]= 0
            elif strong_th>grad_mag>= weak_th:
                ids[i_y, i_x]= 1
            else:
                ids[i_y, i_x]= 2
    
    cv2.imshow('own_canny.png', mag)
    final_edges = np.copy(mag)
    
    idx = 4
    for i in range(idx,ids.shape[0]-idx):
        for j in range(idx,ids.shape[1]-idx):
            if ids[i,j] != 0:
                if np.any(ids[i-idx:i+idx+1,j-idx:j+idx+1] != 1):
                    final_edges[i,j] = 255
                else:
                    final_edges[i,j] = 0
    
    #cv2.imshow('trass',final_edges)
    return mag

# calling the designed function for 
# finding edges 
canny_img = Canny_detector(green_clahe[:,:,1],20,80)

# blurred = cv2.GaussianBlur(green_clahe,(5,5),1.4)
# blurred_edges = cv2.Canny(blurred[:,:,1],20,80)
# cv2.imwrite('blurred_canny_20_80.png',blurred_edges)
# 
# blurred_edges = cv2.Canny(blurred[:,:,1],20,55)
# cv2.imwrite('blurred_canny_20_55.png',blurred_edges)
# 
# blurred_edges = cv2.Canny(blurred[:,:,1],20,35)
# cv2.imwrite('blurred_canny_20_35.png',blurred_edges)


#edges = cv2.Canny(green_clahe[:,:,1],20,80)
#cv2.imshow('canny',edges)
#cv2.imshow('custom_canny',canny_img)
cv2.waitKey(0)

-1

In [21]:


cv2.waitKey(0)

-1

In [39]:
img2draw = get_img()
gray = cv2.cvtColor(img2draw, cv2.COLOR_BGR2GRAY)
cv2.imshow('Original', gray)
cv2.waitKey(0)

-1

In [35]:
thresh = cv2.adaptiveThreshold(green_clahe[:,:,1], 250, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
cv2.imwrite('adaptive_thresh.png',thresh)
cv2.waitKey(0)

-1

Adaptive threshold

In [32]:
img2draw = get_img()
thresh = cv2.adaptiveThreshold(green_clahe[:,:,1], 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('test',img2draw)
cv2.waitKey(0)

-1

Canny

In [88]:
img2draw = get_img()
blurred = cv2.GaussianBlur(green, (5, 5), 0)
#thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,43,13)
edges = cv2.Canny(blurred,25,55)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('canny',edges)
cv2.waitKey(0)

-1

In [38]:
img2draw = get_img()
edges = cv2.Canny(img,100,200)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('canny',edges)
cv2.waitKey(0)

-1

Sobel

In [75]:

sobel_combined = np.uint8(np.absolute(sobel_combined))
img2draw = img.copy()
thresh = cv2.adaptiveThreshold(sobel_combined,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('sobel',img2draw)
cv2.waitKey(0)

-1